In [4]:
import pandas as pd
import numpy as np
from stratified_dataset import ParallelStratifiedSynthesizer
from snsynth.mst import MSTSynthesizer
from snsynth.aim import AIMSynthesizer
from gem_synthesizer import GEMSynthesizer
import dill
from data_utils import get_employment
import itertools
import os
from IPython.display import clear_output
from stratified_dataset import StratifiedDataset
import warnings
warnings.filterwarnings('ignore')

all_data, features, target, group = get_employment()

df = all_data.copy()
df = df.drop(columns=['CIT', 'MIG', 'DEAR', 'DEYE', 'NATIVITY', 'ANC'])


['RELP']


In [85]:
with open('models/MSTSynthesizer_epsilon_1.0_SEX_RAC1P_seed_0.dill', "rb") as file:
    model = dill.load(file)
synth_df = model.sample(len(df))

In [88]:
with open('models/MSTSynthesizer_epsilon_1.0_seed_0.dill', "rb") as file:
    model = dill.load(file)
synth_df = model.sample(len(df))

In [89]:
def get_subgroup_key(group, groupby_cols):
    key = []
    for col in groupby_cols:
        unique_values = group[col].unique()
        if len(unique_values) == 1:
            key.append((col, unique_values[0]))
        else:
            raise ValueError(f"More than one unique value found for column '{col}' in the given group.")
    return tuple(key)

def create_subgroups_dict(X, groupby_cols):
    subgroups = {}
    for _, group in X.groupby(groupby_cols):
        key = get_subgroup_key(group, groupby_cols)
        subgroups[key] = group
    return subgroups

def parity_error_synth_data(X, X_prime, groupby_cols, f, omega):
    subgroups_real = create_subgroups_dict(X, groupby_cols)
    subgroups_synth = create_subgroups_dict(X_prime, groupby_cols)
    print(len(subgroups_real.keys()))
    print(len(subgroups_synth.keys()))
    f_values_real = []
    f_values_synth = []

    # Calculate f and M values for each stratum
    for key, s in subgroups_real.items():
        f_value_real = f(s)
        f_values_real.append(f_value_real)
        
        if key in subgroups_synth:
            f_value_synth = f(subgroups_synth[key])
        else:
            f_value_synth = f(X_prime)

        f_values_synth.append(f_value_synth)

    # Calculate the global f and M values
    f_global = f(X)
    f_synth_global = f(X_prime)

    # Compute the parity error
    beta = omega * abs(f_global - f_synth_global) + sum([abs(t - s) for t, s in zip(f_values_real, f_values_synth)])

    return beta

def f(df, col='ESR'):
    return df[col].mean()



In [90]:
# Define the groupby columns
strata_cols = ['SEX', 'RAC1P']

# Calculate the parity error
omega = 0.0
error = parity_error_synth_data(df, synth_df, strata_cols, f, omega)
print("Parity error:", error)

18
18
Parity error: 1.6401893568599504


In [ ]:
import numpy as np
import random

def random_queries(num_queries, columns, min_values, max_values):
    queries = []
    for _ in range(num_queries):
        query = {}
        for col in columns:
            query[col] = (random.uniform(min_values[col], max_values[col]), random.uniform(min_values[col], max_values[col]))
        queries.append(query)
    return queries

def query_result(df, query):
    mask = np.full(df.shape[0], True, dtype=bool)
    for col, (min_val, max_val) in query.items():
        mask &= (df[col] >= min_val) & (df[col] <= max_val)
    return df[mask].shape[0]

def f(X, X_prime, num_queries=10, columns=None):
    if columns is None:
        columns = X.columns
    
    min_values = {col: min(X[col].min(), X_prime[col].min()) for col in columns}
    max_values = {col: max(X[col].max(), X_prime[col].max()) for col in columns}
    
    queries = random_queries(num_queries, columns, min_values, max_values)
    errors = []
    for query in queries:
        result_real = query_result(X, query)
        result_synth = query_result(X_prime, query)
        error = abs(result_real - result_synth)
        if result_real > 0:
            error /= result_real
        errors.append(error)
    return np.mean(errors)